In [47]:
import os
import pandas as pd
import numpy as np
import mygene
import gseapy as gp
import gseapy.plot as gseaplot
import matplotlib.pyplot as plt

from gseapy import enrichment_map
import networkx as nx

# %matplotlib inline
# %config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt

from gseapy import Biomart
bm = Biomart()

from gseapy import gseaplot, heatmap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
tar4 = pd.read_csv('targets4.csv')
tar12 = pd.read_csv('targets12.csv')
tar24 = pd.read_csv('targets24.csv')

In [49]:
tar24

,Unnamed: 0,0,1,2
0,395561,cfa-miR-429,SYN3,84.065900
1,395562,cfa-miR-429,MECOM,83.408206
2,395563,cfa-miR-429,CEP44,82.414200
3,395573,cfa-miR-429,PPP3CA,87.757566
4,395576,cfa-miR-429,ATVR1,93.518910
...,...,...,...,...
4243,1579601,gga-let-7a-3p,Rexo1,83.486742
4244,1579603,gga-let-7a-3p,Cbln2,91.088916
4245,1579606,gga-let-7a-3p,Asph,85.016824
4246,1579615,gga-let-7a-3p,Dennd6a,80.785400


In [50]:
set(tar24['0'])

{'cfa-miR-429',
 'gga-let-7a-3p',
 'gga-miR-15b-5p',
 'gga-miR-21-5p',
 'gga-miR-29b-3p',
 'gga-miR-429-3p',
 'gga-miR-429-5p'}

In [51]:
set(tar4['0'])

{'cfa-miR-429',
 'gga-miR-16-5p',
 'gga-miR-21-5p',
 'gga-miR-29a-3p',
 'gga-miR-29c-3p',
 'gga-miR-429-3p',
 'gga-miR-429-5p'}

In [52]:
set(tar12['0'])

{'cfa-miR-429',
 'gga-miR-16-5p',
 'gga-miR-21-5p',
 'gga-miR-27b-3p',
 'gga-miR-29a-3p',
 'gga-miR-29c-3p',
 'gga-miR-429-3p',
 'gga-miR-429-5p'}

In [53]:
genes429 = tar12[tar12['0'].str.contains('429')]['1']

In [60]:
genes429

0         SYN3
1        MECOM
2        CEP44
3       PPP3CA
4        ATVR1
         ...  
3129      Nme7
3130       Wac
3131     Tcte1
3132     Lmtk2
3133     Otud4
Name: 1, Length: 1434, dtype: object

In [54]:
def enrich_genes(top_ranked_genes):

    enr_Reactome_up = gp.enrichr(gene_list=top_ranked_genes,
                                 gene_sets=['Reactome_2022'],
                                 organism='Human', 
                                 cutoff=0.5)
    
    enr_GOBP_up = gp.enrichr(gene_list=top_ranked_genes,
                             gene_sets=['GO_Biological_Process_2021'],
                             organism='Human', 
                             cutoff=0.5)
    
    enr_GOMF_up = gp.enrichr(gene_list=top_ranked_genes ,
                             gene_sets=['GO_Molecular_Function_2021'],
                             organism='Human', 
                             cutoff=0.5)
    
    
    # Write results to CSV files
    #file_name = os.path.basename(file_path).split('.')[0]
    #enr_Reactome_up.results.to_csv(f'{file_name}_enr_Reactome.csv')
    
    return enr_Reactome_up.results, enr_GOBP_up.results, enr_GOMF_up.results

In [55]:
# get the enriched pathways from the datasets reactome, molecular function, biological process

In [56]:
enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(genes429)

In [57]:
enr_Reactome_up.head(10)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2022,RAC1 GTPase Cycle R-HSA-9013149,27/178,1.748532e-07,0.000229,0,0,3.532534,54.963817,CYFIP1;NCKAP1;DOCK5;DOCK4;FAM13B;GARRE1;RASGRF...
1,Reactome_2022,Signal Transduction R-HSA-162582,172/2465,8.212072e-07,0.000537,0,0,1.548836,21.703053,SH3GL3;CYFIP1;NCKAP1;AHCYL1;FAM13B;PLXND1;DGKB...
2,Reactome_2022,RHO GTPase Cycle R-HSA-9012999,44/441,6.805438e-06,0.002697,0,0,2.200201,26.177525,CYFIP1;NCKAP1;DOCK5;DOCK4;FAM13B;PLXND1;GARRE1...
3,Reactome_2022,Heme Signaling R-HSA-9707616,11/45,8.241032e-06,0.002697,0,0,6.325317,74.046590,NCOA2;BRIP1;CREB1;NCOR1;TBL1XR1;NRIP1;RORA;LIN...
4,Reactome_2022,"Signaling By Rho GTPases, Miro GTPases And RHO...",57/660,2.566666e-05,0.005804,0,0,1.881861,19.891868,CYFIP1;NCKAP1;DOCK5;AHCTF1;DOCK4;FAM13B;PLXND1...
5,Reactome_2022,Circadian Clock R-HSA-400253,13/69,2.660534e-05,0.005804,0,0,4.542717,47.854793,NCOA2;CPT1A;RORA;CSNK1D;LIN9;PPP1CB;CREB1;NCOR...
6,Reactome_2022,Muscle Contraction R-HSA-397014,24/196,3.525470e-05,0.006593,0,0,2.744963,28.143862,TMOD1;GUCY1A2;KCNK7;AHCYL1;GUCY1B1;TPM3;TPM1;I...
7,Reactome_2022,Signaling By Rho GTPases R-HSA-194315,55/644,4.808684e-05,0.007868,0,0,1.856391,18.457170,CYFIP1;NCKAP1;DOCK5;AHCTF1;DOCK4;FAM13B;PLXND1...
8,Reactome_2022,RHOA GTPase Cycle R-HSA-8980692,19/147,1.085780e-04,0.012428,0,0,2.911718,26.578283,PLEKHG4;STARD13;ARHGEF15;SOWAHC;CAV1;RASGRF2;E...
9,Reactome_2022,Signaling By Receptor Tyrosine Kinases R-HSA-9...,44/496,1.153505e-04,0.012428,0,0,1.926769,17.471050,CYFIP1;NCKAP1;NRP1;SH3GL3;AHCYL1;SH3KBP1;NAB1;...


In [58]:
genes27b = tar12[tar12['0'].str.contains('27b')]['1']

In [59]:
enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(genes27b)
enr_Reactome_up.head(10)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2022,Competing Endogenous RNAs (ceRNAs) Regulate PT...,4/10,0.000444,0.159858,0,0,16.033417,123.772329,VAPA;AGO4;AGO1;TNRC6B
1,Reactome_2022,Signaling By Receptor Tyrosine Kinases R-HSA-9...,36/496,0.000467,0.159858,0,0,1.914428,14.682128,GABRB2;HDAC2;AHCYL1;SH3KBP1;SRF;PTPRJ;PIK3CB;P...
2,Reactome_2022,Signaling By ALK R-HSA-201556,6/26,0.000472,0.159858,0,0,7.227889,55.358559,HDAC2;PIK3CA;MYC;PIK3CB;PRDM1;HIF1A
3,Reactome_2022,Regulation Of Signaling By CBL R-HSA-912631,5/18,0.000567,0.159858,0,0,9.258276,69.202767,LYN;PIK3CA;RAPGEF1;FYN;PIK3CB
4,Reactome_2022,Regulation Of PTEN mRNA Translation R-HSA-8943723,4/11,0.000676,0.159858,0,0,13.742213,100.315929,VAPA;AGO4;AGO1;TNRC6B
5,Reactome_2022,Signal Transduction R-HSA-162582,129/2465,0.000832,0.160926,0,0,1.383601,9.812535,ANKLE2;AHCYL1;RTKN;KDM1B;RORA;PRDM1;CXCL16;ACT...
6,Reactome_2022,RNA Polymerase III Abortive And Retractive Ini...,7/40,0.000952,0.160926,0,0,5.113589,35.573738,GTF3C2;NFIX;POU2F1;SNAPC1;NFIA;NFIB;POLR3K
7,Reactome_2022,Regulation Of MECP2 Expression And Activity R-...,6/31,0.001272,0.174910,0,0,5.780804,38.541668,MECP2;HDAC2;AGO4;AGO1;SLC6A4;TNRC6B
8,Reactome_2022,Nervous System Development R-HSA-9675108,37/545,0.001331,0.174910,0,0,1.779450,11.783644,SEMA5A;ROBO3;GSK3B;HDAC2;SH3KBP1;SEMA3A;PIK3CB...
9,Reactome_2022,Role Of LAT2/NTAL/LAB On Calcium Mobilization ...,4/14,0.001861,0.196356,0,0,9.618045,60.463153,LYN;PIK3CA;FYN;PIK3CB
